## Кодування Гафмена

### Вхідний текст: AAAABBBBCCCCDDDDAAAABBBBCCCCDDDD

---

### Частоти символів:
| Символ | Кількість |
|--------|-----------|
| A      | 8         |
| B      | 8         |
| C      | 8         |
| D      | 8         |

> Усі символи мають однакову частоту — це дає **симетричне дерево**.

---

### Побудова дерева

Для рівномірних частот дерево Гафмена створює двійкові коди довжиною 2 біта. Наприклад:

| Символ | Код |
|--------|-----|
| A      | 00  |
| B      | 01  |
| C      | 10  |
| D      | 11  |

> Це відповідає найкращому випадку.

---

### Закодований текст

Закодований рядок має довжину **128 біт**.

- Кількість символів: 32
- Кожен символ кодується 2 бітами (оптимально)

---

### Оцінка ефекту від стиснення

| Випадок         | Розмір (в бітах) | Коментар                    |
|------------------|------------------|-----------------------------|
| ASCII (8 біт)    | 256              | Неоптимальний               |
| Гафмен           | 128              | Оптимальне стиснення        |
| Найгірший випадок| 160-256          | Якщо символи мали різні коди довжини |

> Таким чином, ми зекономили 50% пам’яті порівняно з ASCII.

---

## Контрольні питання

1. **Що таке кодування Гафмена та як воно працює?**  
   Це алгоритм безвтратного стиснення, який присвоює коротші коди символам з більшою частотою. Будується дерево, де листками є символи, а шляхи до них — коди.

2. **Як визначається оптимальний двійковий код?**  
   Алгоритм будує бінарне дерево за принципом злиття найменш вірогідних символів, забезпечуючи мінімальну середню довжину коду.

3. **Переваги над іншими методами:**  
   - Оптимальний для відомих частот
   - Простий у реалізації
   - Гарантовано без втрат

4. **Як відбувається декодування?**  
   За допомогою дерева: кожен біт веде ліворуч (0) або праворуч (1), і при досягненні листа ми відновлюємо символ.

5. **Можливі недоліки:**  
   - Не працює ефективно при малій кількості унікальних символів
   - Необхідно зберігати таблицю або дерево для декодування

6. **Для чого дерево?**  
   Дерево дозволяє уникнути двозначності коду та забезпечує однозначне декодування.


In [1]:
from collections import Counter
import heapq
import matplotlib.pyplot as plt
import networkx as nx

# Вхідний текст
text = "AAAABBBBCCCCDDDDAAAABBBBCCCCDDDD"

# Частоти символів
freq = Counter(text)
print("Частоти символів:", freq)

# Побудова дерева Гафмена
heap = [[weight, [symbol, ""]] for symbol, weight in freq.items()]
heapq.heapify(heap)

while len(heap) > 1:
    lo = heapq.heappop(heap)
    hi = heapq.heappop(heap)
    for pair in lo[1:]:
        pair[1] = '0' + pair[1]
    for pair in hi[1:]:
        pair[1] = '1' + pair[1]
    heapq.heappush(heap, [lo[0] + hi[0]] + lo[1:] + hi[1:])

# Отримання кодів
huff_codes = sorted(heap[0][1:], key=lambda p: (len(p[-1]), p))
print("\nКоди Гафмена:")
for symbol, code in huff_codes:
    print(f"{symbol}: {code}")

# Закодувати текст
encoded_text = ''.join(dict(huff_codes)[char] for char in text)
print("\nЗакодований текст:", encoded_text)
print("Довжина закодованого тексту (в бітах):", len(encoded_text))

# Для оцінки:
original_bits = len(text) * 8  # 8 біт на символ в ASCII
best_case = len(text) * 2      # якщо бітів на символ = 2 (т.к. 4 символи = 2 біта)
worst_case = len(text) * len(max([code for _, code in huff_codes], key=len))

print("\nОцінка ефективності:")
print(f"Початкова довжина (ASCII): {original_bits} біт")
print(f"Стиснуто (Гафмен): {len(encoded_text)} біт")
print(f"Найгірший випадок (однакові довжини): {worst_case} біт")
print(f"Найкращий випадок (по 2 біта на символ): {best_case} біт")


Частоти символів: Counter({'A': 8, 'B': 8, 'C': 8, 'D': 8})

Коди Гафмена:
A: 00
B: 01
C: 10
D: 11

Закодований текст: 0000000001010101101010101111111100000000010101011010101011111111
Довжина закодованого тексту (в бітах): 64

Оцінка ефективності:
Початкова довжина (ASCII): 256 біт
Стиснуто (Гафмен): 64 біт
Найгірший випадок (однакові довжини): 64 біт
Найкращий випадок (по 2 біта на символ): 64 біт
